<a href="https://colab.research.google.com/github/The-cheater/Deep_Learning_Models/blob/main/MMCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
import random
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Unzip GAF and MTF images from Drive (adjust paths if necessary)
gaf_zip = "/content/drive/MyDrive/dataset/GAF_Images_224x224.zip"
mtf_zip = "/content/drive/MyDrive/dataset/MTF_Images_224x224.zip"

# Create directories to extract to
gaf_extract_path = "/content/GAF_Images_224x224"
mtf_extract_path = "/content/MTF_Images_224x224"
os.makedirs(gaf_extract_path, exist_ok=True)
os.makedirs(mtf_extract_path, exist_ok=True)


with zipfile.ZipFile(gaf_zip, 'r') as zip_ref:
    zip_ref.extractall(gaf_extract_path)

with zipfile.ZipFile(mtf_zip, 'r') as zip_ref:
    zip_ref.extractall(mtf_extract_path)

# %%
class GAFMTFGenerator(Sequence):
    def __init__(self, gaf_dir, mtf_dir, batch_size=32, image_size=(224, 224), shuffle=True):
        self.gaf_dir = gaf_dir
        self.mtf_dir = mtf_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle

        # Assuming the class directories are within a specific nested folder
        # like /content/GAF_Images_224x224/GAF_Images_224x224/Left Stride Interval/Class1
        # We need to navigate to the 'Left Stride Interval' folder first
        nested_gaf_base = os.path.join(gaf_dir, os.path.basename(gaf_dir), "Left Stride Interval")
        nested_mtf_base = os.path.join(mtf_dir, os.path.basename(mtf_dir), "Left Stride Interval")

        # Now list the actual class directories within this base path
        # Add a check to ensure the nested base paths exist
        if not os.path.exists(nested_gaf_base):
            raise FileNotFoundError(f"Nested GAF directory not found: {nested_gaf_base}")
        if not os.path.exists(nested_mtf_base):
             raise FileNotFoundError(f"Nested MTF directory not found: {nested_mtf_base}")

        self.class_names = sorted(os.listdir(nested_gaf_base))
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(self.class_names)

        self.samples = []
        for label in self.class_names:
            # Construct the correct class path relative to the 'Left Stride Interval' base
            gaf_class_path = os.path.join(nested_gaf_base, label)
            mtf_class_path = os.path.join(nested_mtf_base, label)

            if os.path.exists(gaf_class_path) and os.path.exists(mtf_class_path):
                for fname in os.listdir(gaf_class_path):
                    if fname.endswith(".png") and os.path.exists(os.path.join(mtf_class_path, fname)):
                        self.samples.append((os.path.join(gaf_class_path, fname),
                                             os.path.join(mtf_class_path, fname),
                                             label))
            else:
                 print(f"Warning: Class directory pair not found for label '{label}': {gaf_class_path} or {mtf_class_path}")


        print(f"📦 Total image pairs loaded: {len(self.samples)}")
        if self.shuffle:
            random.shuffle(self.samples)

    def __len__(self):
        # Ensure length is at least 1 if samples exist to avoid division by zero
        if not self.samples:
            return 0
        return len(self.samples) // self.batch_size

    def __getitem__(self, idx):
        # Add error handling for out of bounds index
        if idx * self.batch_size >= len(self.samples):
            raise IndexError("Index out of bounds for data generator")

        batch = self.samples[idx * self.batch_size : (idx + 1) * self.batch_size]
        gaf_batch, mtf_batch, labels = [], [], []

        for gaf_path, mtf_path, label in batch:
            gaf_img = img_to_array(load_img(gaf_path, target_size=self.image_size)) / 255.0
            mtf_img = img_to_array(load_img(mtf_path, target_size=self.image_size)) / 255.0
            gaf_batch.append(gaf_img)
            mtf_batch.append(mtf_img)
            labels.append(label)

        gaf_batch = np.array(gaf_batch)
        mtf_batch = np.array(mtf_batch)
        labels = self.label_encoder.transform(labels)
        labels = tf.keras.utils.to_categorical(labels, num_classes=len(self.class_names))

        # Return a dictionary for multi-input model
        return {"gaf_input": gaf_batch, "mtf_input": mtf_batch}, labels


    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.samples)

# %%
def build_mmcnn(input_shape=(224, 224, 3), num_classes=4):
    # GAF branch
    gaf_input = Input(shape=input_shape, name='gaf_input')
    gaf_conv1 = Conv2D(32, (3, 3), activation='relu')(gaf_input)
    gaf_pool1 = MaxPooling2D((2, 2))(gaf_conv1)
    gaf_conv2 = Conv2D(64, (3, 3), activation='relu')(gaf_pool1)
    gaf_pool2 = MaxPooling2D((2, 2))(gaf_conv2)
    gaf_flatten = Flatten()(gaf_pool2)

    # MTF branch
    mtf_input = Input(shape=input_shape, name='mtf_input')
    mtf_conv1 = Conv2D(32, (3, 3), activation='relu')(mtf_input)
    mtf_pool1 = MaxPooling2D((2, 2))(mtf_conv1)
    mtf_conv2 = Conv2D(64, (3, 3), activation='relu')(mtf_pool1)
    mtf_pool2 = MaxPooling2D((2, 2))(mtf_conv2)
    mtf_flatten = Flatten()(mtf_pool2)

    # Combine branches
    combined = concatenate([gaf_flatten, mtf_flatten])

    # Dense layers
    dense1 = Dense(128, activation='relu')(combined)
    output = Dense(num_classes, activation='softmax')(dense1)

    # Create model
    model = Model(inputs=[gaf_input, mtf_input], outputs=output)

    return model

# %%
# Instantiate the model
# Make sure to set num_classes to the actual number of classes you have
# Based on your path, it seems you might have classes like 'als', 'control', etc.
# Update num_classes if you have more or less than 4 classes
model = build_mmcnn(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# %%
# Instantiate the data generator
# Use the top-level unzipped directories
gaf_path = "/content/GAF_Images_224x224"
mtf_path = "/content/MTF_Images_224x224"

train_gen = GAFMTFGenerator(gaf_path, mtf_path, batch_size=16)

# Train the model
model.fit(train_gen, epochs=20)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📦 Total image pairs loaded: 256


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.2596 - loss: 14.2138
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.5468 - loss: 1.1555
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.7869 - loss: 0.6542
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.9255 - loss: 0.3538
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.9843 - loss: 0.0743
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 1.0000 - loss: 0.0237
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 1.0000 - loss: 0.0051
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 1.0000 - loss: 7.3660e-04
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 1.0000 - loss: 4.4712e-04
Epoch 11/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 1.0000 - loss: 2.7796e-04
Epoch 12/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
from google.colab import files

# Upload both GAF and MTF images
print("📂 Please upload the GAF and MTF images (must be .png)")
uploaded = files.upload()


📂 Please upload the GAF and MTF images (must be .png)


Saving hunt1_x_shifted_gaf.png to hunt1_x_shifted_gaf (1).png
Saving hunt1_x_shifted_mtf.png to hunt1_x_shifted_mtf.png


In [8]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Detect which file is GAF and which is MTF based on filename
gaf_img_path = [f for f in uploaded if 'gaf' in f.lower()][0]
mtf_img_path = [f for f in uploaded if 'mtf' in f.lower()][0]

# Load and preprocess
def preprocess_image(path):
    img = load_img(path, target_size=(224, 224))
    arr = img_to_array(img) / 255.0
    return np.expand_dims(arr, axis=0)

gaf_input = preprocess_image(gaf_img_path)
mtf_input = preprocess_image(mtf_img_path)


In [9]:
# Predict using your trained model
prediction = model.predict({"gaf_input": gaf_input, "mtf_input": mtf_input})
predicted_index = np.argmax(prediction)

# Use class labels from your generator
class_labels = train_gen.class_names
predicted_label = class_labels[predicted_index]

print("✅ Predicted Disease Class:", predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ Predicted Disease Class: hunt


In [10]:
for label, prob in zip(class_labels, prediction[0]):
    print(f"{label}: {prob:.2%}")


als: 0.00%
control: 0.00%
hunt: 100.00%
park: 0.00%
